In [1]:
#Access files from drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#View ADNI Merge File
import pandas as pd

file = 'drive/MyDrive/Colab Notebooks/ADNI AD detection/ADNIMERGE.csv'
subs = ['PTID', 'Month', 'IMAGEUID', 'DX']
df = pd.read_csv(file).dropna(subset = subs)[subs] 
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (19,20,21,104,105,106) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,PTID,Month,IMAGEUID,DX
0,011_S_0002,0,35475.0,CN
1,011_S_0003,0,32237.0,Dementia
2,011_S_0003,6,31863.0,Dementia
3,011_S_0003,12,35576.0,Dementia
4,011_S_0003,24,88252.0,Dementia
...,...,...,...,...
15780,024_S_6472,36,1453675.0,CN
15795,003_S_4644,108,1477561.0,CN
15799,037_S_7006,0,1514928.0,CN
15919,082_S_5282,84,1499323.0,MCI


In [3]:
#Return image ids of patients with AD and CN diagnosis
def return_list_of_patients(dataframe):
  list_of_patients = []
  patient_ids = []
  new_df = pd.DataFrame()
  for i in range(len(dataframe.index)):
    if dataframe.iat[i,3] == 'CN' or dataframe.iat[i,3] == 'Dementia':
      if not(dataframe.iat[i,0] in patient_ids):
        patient_ids.append(dataframe.iat[i,0])
        list_of_patients.append(int(dataframe.iat[i,2]))
        row = df.iloc[[i]]
        new_df = new_df.append(row)
  return list_of_patients, patient_ids,new_df
list_of_patients, patient_ids,new_df = return_list_of_patients(df)
print(list_of_patients)

[35475, 32237, 32246, 59366, 32270, 59375, 33066, 32306, 32332, 32409, 391070, 788060, 119180, 33074, 34607, 118799, 387866, 120056, 33086, 119273, 35486, 47898, 119795, 119186, 74240, 118832, 119188, 388019, 119062, 544727, 59429, 118674, 118677, 33105, 118686, 119274, 119067, 119190, 68125, 65989, 118869, 118874, 118970, 119678, 119277, 119227, 388925, 35841, 119288, 118969, 119300, 59456, 64046, 148873, 140762, 119307, 75111, 388923, 130231, 119194, 49487, 34353, 119322, 35629, 119070, 63784, 35671, 142353, 74217, 59484, 64663, 118971, 73584, 119079, 39533, 33489, 544732, 119325, 39810, 35569, 65752, 96321, 119196, 78876, 32003, 119711, 119715, 384777, 101510, 74514, 95674, 119717, 81596, 40292, 66608, 75669, 130237, 60600, 105747, 119376, 171864, 67152, 72128, 78682, 32855, 49735, 119381, 119383, 102476, 39108, 82125, 143462, 64948, 119385, 176231, 35647, 119198, 109722, 34362, 34384, 124897, 119390, 80424, 119392, 66428, 118779, 123389, 775701, 118671, 721319, 119328, 119726, 1197

In [6]:
#imports
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import os
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

data_parent_dir = 'drive/MyDrive/Colab Notebooks/ADNI AD detection/AD_Detection/ADNI'

#Create iterative list of images files
image_file_dir = []
files = os.listdir(data_parent_dir)
for i in files:
  file = os.path.join(data_parent_dir, i)
  for j in range(4):
    next_file = os.listdir(file)
    file = os.path.join(file, next_file[len(next_file) - 1])
  image_file_dir.append(file)
  print(file)
print(image_file_dir)

drive/MyDrive/Colab Notebooks/ADNI AD detection/AD_Detection/ADNI/073_S_4559/MT1__N3m/2016-06-30_12_13_28.0/S516862/reorient.nii
drive/MyDrive/Colab Notebooks/ADNI AD detection/AD_Detection/ADNI/073_S_0909/MPR__GradWarp__B1_Correction__N3__Scaled/2009-12-07_14_45_09.0/S77038/ADNI_073_S_0909_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20100301133616796_S77038_I167021.nii
drive/MyDrive/Colab Notebooks/ADNI AD detection/AD_Detection/ADNI/073_S_4393/MT1__GradWarp__N3m/2011-11-22_11_01_16.0/S130463/ADNI_073_S_4393_MR_MT1__GradWarp__N3m_Br_20111206103537363_S130463_I270031.nii
drive/MyDrive/Colab Notebooks/ADNI AD detection/AD_Detection/ADNI/073_S_4382/MT1__GradWarp__N3m/2011-11-15_13_00_58.0/S129598/ADNI_073_S_4382_MR_MT1__GradWarp__N3m_Br_20111121113914030_S129598_I267773.nii
drive/MyDrive/Colab Notebooks/ADNI AD detection/AD_Detection/ADNI/073_S_0312/MPR__GradWarp__B1_Correction__N3__Scaled_2/2006-05-31_13_21_45.0/S15079/ADNI_073_S_0312_MR_MPR__GradWarp__B1_Correction__N3__Scaled_2_Br_

In [7]:

#Function for viewing the image
def show_image(image_file_index, layer):
  file = image_file_dir[image_file_index]
  load = nib.load(file).get_data()
  print(load.shape)
  show_load = load[:,:,layer]
  plt.imshow(show_load, cmap = 'gray')
  plt.show()
  print(file)

#Slider to view different layers
interact(show_image, layer=(0,160-1), image_file_index=999)

interactive(children=(IntSlider(value=999, description='image_file_index', max=2997, min=-999), IntSlider(valu…

<function __main__.show_image>

In [8]:
#Get labels
labels = []
new_df
for i in files:
  row = new_df.loc[df['PTID'] == i]
  val = row.iat[0,3]
  if val == 'Dementia':
    labels.append(1)
  else:
    labels.append(0)